### FUNÇÃO DE TRANSFERENCIA PARA SIMULAÇÃO DE RUÍDOS DE VAZAMENTO

In [4]:
%matplotlib qt 

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.signal import lfilter

In [6]:
def butter_bandpass(lowcut, highcut, fs, order=3):
    """
    Implementação do filtro butterworth passa-banda.
    """

#oi gabriel
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype="band")
    return b, a

In [7]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    """
    Filtro butterworth passa-banda.
    """

    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [8]:
# Função de transferência H conforme a Equação (6)
def H(d, omega, beta, c, A_n, n):
    """
    Calcula a função de resposta de frequência H(d, omega).
    
    Parâmetros:
    - d: distância do sensor ao vazamento (m)
    - omega: frequência angular (rad/s)
    - beta: fator de atenuação (1/m)
    - c: velocidade de propagação do ruído (m/s)
    - A_n: ganho associado ao tipo de sensor
    - n: ordem do sensor (0: hidrofones, 1: geofones, 2: acelerômetros)
    
    Retorna:
    - H: valor complexo da FRF
    """
    return A_n * (1j * omega)**n * np.exp(-omega * beta * d) * np.exp(-1j * omega * d / c)

In [9]:
def plot_correlation(corr, lags, name = None):
    
    max_lag = int(lags[np.argmax(corr)])

    plt.plot(lags, corr, label=f'Max lag = {max_lag}ms')
    plt.ylabel('Intensidade - correlação')
    plt.legend()
    plt.xlabel('Lag(ms)')
    if name:
        plt.savefig(name)
    plt.show()

In [10]:
def norm_correlate(x, y):
    """
    Calcula a correlação cruzada (completa) entre dois sinais.
    """

    n = min(len(x), len(y))
    c = signal.correlate(x / np.std(x), y / np.std(y), mode="full")
    return c / n


In [11]:
def correlate(audio1, audio2, sr):
    """
    Recebe uma instância da correlação, realiza os cálculos e salva os mesmos no banco de dados.
    O argumento lag_interval é utilizado para limitar a área máxima da correlação.
    """

    corr = norm_correlate(audio1, audio2)

    corr = corr.tolist()

    lags = signal.correlation_lags(len(audio1), len(audio2))


    lags_ms = [(i / sr) * 1000 for i in lags]  # A sample rate do sinal de áudio é sempre a mesma que a do rádio.

    max_lag = int(lags_ms[np.argmax(corr)])

    results_dict = {}

    results_dict["lags_ms"] = lags_ms
    results_dict["max_lag"] = max_lag
    results_dict["result"] = corr

    return results_dict

In [12]:
# Parâmetros do sistema
d = 70  # Distância do sensor ao vazamento (m)
beta = 1.0e-4  # Fator de atenuação (1/m)
c = 356  # Velocidade de propagação (m/s)

# Parâmetros específicos para hidrofones (n = 0)
n_hydrophone = 0
A_hydrophone = 1  # Ganho específico do hidrofone (sem unidades)

# Parâmetros específicos para geofones (n = 1)
n_geophone = 1
A_geophone = 1  # Exemplo: substitua com valores reais calculados

# Parâmetros específicos para acelerômetros (n = 2)
n_accelerometer = 2
A_accelerometer = 1  # Exemplo: substitua com valores reais calculados




# Geração de frequências e cálculo de H
fs = 1960  # Frequência de amostragem (Hz)


dt = 1/fs
t = np.arange(0,5,dt)

frequencies = np.fft.rfftfreq(len(t), 1 / fs)  # Frequências de Fourier
omega = 2 * np.pi * frequencies  # Frequências angulares (rad/s)

# Calculando as FRFs
H_hydrophone = H(d, omega, beta, c, A_hydrophone, n_hydrophone)
H_geophone = H(d, omega, beta, c, A_geophone, n_geophone)
H_accelerometer = H(d, omega, beta, c, A_accelerometer, n_accelerometer)


In [13]:
# Visualização dos módulos
plt.figure(figsize=(10, 5))
plt.plot(frequencies, np.abs(H_hydrophone)/max(np.abs(H_hydrophone)), label="Hydrophone (n=0)")
plt.plot(frequencies, np.abs(H_geophone)/max(np.abs(H_geophone)), label="Geofone (n=1)")
plt.plot(frequencies, np.abs(H_accelerometer)/max(np.abs(H_accelerometer)), label="Acelerômetro (n=2)")
plt.title("Módulo da Função de Resposta de Frequência (FRF)")
plt.xlabel("Frequência [Hz]")
plt.ylabel("|H(d, ω)|")
plt.grid()
plt.legend()

### Gerando um sinal aleatório para simular o ruído de vazamento

In [14]:
sig = np.random.randn(len(t))

sig_fft = np.fft.rfft(sig)

In [15]:
max(sig)

3.7768835756617745

In [16]:
H_response1 = H(70, omega, beta, c, A_hydrophone, n_hydrophone)
H_response2 = H(200, omega, beta, c, A_hydrophone, n_hydrophone)

#aplicando a FRF ao ruído
sig1 = np.fft.irfft(H_response1 * sig_fft)
sig2 = np.fft.irfft(H_response2 * sig_fft)


In [17]:
f, Pxy = signal.csd(sig1, sig2, fs, nperseg=2*fs)

# Extrair o ângulo de fase em radianos
phase_rad = np.angle(Pxy)

# Desembrulhar a fase para evitar saltos bruscos
unwrapped_phase_rad = np.unwrap(phase_rad)

# Converter para graus
unwrapped_phase_deg = np.degrees(unwrapped_phase_rad)

plt.xlim([0, 1000])
plt.plot(f, unwrapped_phase_deg)
plt.title('Unwrapped Phase Angle (graus)')
plt.xlabel('Frequência [Hz]')
plt.ylabel('Fase [graus]')
plt.show()

In [18]:
f, Pxy = signal.coherence(sig1, sig2, fs, nperseg=2*fs)

# Adicionar linha no eixo y=0 (linha horizontal)
plt.axhline(0, color='black',linewidth=1)

# Adicionar linha no eixo x=0 (linha vertical)
plt.axvline(0, color='black',linewidth=1)

plt.xlim([0, 1000])
plt.plot(f, 10*np.log10(Pxy))
plt.title('Cross power spectrum density')
plt.xlabel('Frequência [Hz]')
plt.ylabel('Intensidade dB')
plt.show()

In [19]:
s1 = butter_bandpass_filter(sig1, 1, 140, fs)
s2 = butter_bandpass_filter(sig2, 1, 140, fs)

results = correlate(s1, s2, fs)
plot_correlation(results['result'], results["lags_ms"])